# Splunk App for Data Science and Deep Learning - Notebook Template 

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DSDL) - formerly known as the Deep Learning Toolkit for Splunk (DLTK). Find more examples and information in the app and on the [DSDL splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import re
#from hashlib import md5
import hashlib
from base64 import encode
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

## push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("hashing_encoder_single_ua")

In [ ]:
df

In [ ]:
param

## Stage 2 - create and initialize a model

In [ ]:

def init(df,param):
    model = {} 
    
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [ ]:
def fit(model,df,param):
    return df

In [ ]:
returns = fit(model,df,param)
print(returns)

## Stage 4 - apply the model

In [ ]:
def apply(model,df,param):
    
    field_name=param['feature_variables'][0]
    
    try:
        vector_length=int(param['options']['params']['vector_length'])
    except:
        vector_length=8
    
    try:
        regex=param['options']['params']['tokenizing_regex']
    except:
        regex=r'[^\w\s]'
    
    # define the regex pattern used for tokenisation
    pattern = re.compile(regex)
    # split results up using the regex pattern to create a list of tokens
    df['token_list'] = df[field_name].astype(str).apply(lambda x: pattern.split(x))
    # apply strip to each list item (token) to remove leading and trailing whitespace
    df['token_list'] = df['token_list'].apply(lambda x: [item.strip() for item in x])
    df['token_list'] = df['token_list'].apply(lambda row: [item for item in row if item != ''])
    
    # Now that we have the tokens, we need to hash them and apply hashing trick to create an embedding vector
    # define length of enbedding vector

    # initialise a new vector of length embedding_len, defined as a new field in the query results
    df['embedding'] = df.apply(lambda row: [0] * vector_length, axis=1)

    # convert tokens to indexes into embedding array
    df['index_list'] = df['token_list'].apply(lambda token_list: [int(hashlib.md5(token.encode()).hexdigest(), 16) % (vector_length) for token in token_list])
    
    # 
    # Note: this still needs vectorizing
    for row in range(len(df['index_list'])):
        for col in df['index_list'][row]:
            df['embedding'][row][col] += 1
            
    output_df = df['embedding'].apply(pd.Series)
            
    return output_df

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
training_instance = apply(model,df,param)
print(training_instance)
#training_instance[583:]

#df['useragent_2'][0]

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [ ]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [ ]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

In [ ]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code